In [1]:
from langchain.agents import load_tools
from langchain.agents import initialize_agent
from langchain_ollama import ChatOllama
import os 

from dotenv import load_dotenv
load_dotenv()

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [19]:
from langchain.agents import create_tool_calling_agent
from langchain.prompts import ChatPromptTemplate
from langchain.agents import AgentExecutor

prompt = ChatPromptTemplate.from_template('You are a helpful assistant: {query} agent_scratchpad:{agent_scratchpad}')

llama_llm = ChatOllama(model='llama3.2:1b', base_url='http://localhost:11434', temperature=0)

tools = load_tools(["ddg-search", "openweathermap-api"], llm = llama_llm)

agent = create_tool_calling_agent(llama_llm, tools, prompt)

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True, return_intermediate_steps=True)

output = agent_executor.invoke(
        {
    "query": "1. Who is current IIT Madras Director? 2. Where was he born? 3. What is the current weather there?"
})



> Entering new AgentExecutor chain...

Invoking: `duckduckgo_search` with `{'query': 'IIT Madras Director'}`


Indian Institute of Technology (IIT), Madras bagging the top rank for the sixth time in a row is a validation of the institute's dedication to producing world-class talent and driving impactful ... September 23, 2024. Listen to this story. The Shakti-man of India, Veezhinathan Kamakoti is the director of IIT Madras, and has kept the institution at the top of all universities in the nation. The genius behind India's first microprocessor, Shakti, has been relentlessly working to strengthen the nation's semiconductor capabilities and ... IIT- Madras director Prof V Kamakoti, who was in the national capital to attend the event, spoke to News18, on what makes the institute stick to the top every year, what it is that it strives to improve upon next, why it wants to beat IISc in the rankings, a positive trend in core jobs and why AI will create more jobs in the future. ... IIT Mad

In [37]:
for result in output['intermediate_steps']:
    print(result[0].tool, result[1], '\n')

duckduckgo_search Indian Institute of Technology (IIT), Madras bagging the top rank for the sixth time in a row is a validation of the institute's dedication to producing world-class talent and driving impactful ... September 23, 2024. Listen to this story. The Shakti-man of India, Veezhinathan Kamakoti is the director of IIT Madras, and has kept the institution at the top of all universities in the nation. The genius behind India's first microprocessor, Shakti, has been relentlessly working to strengthen the nation's semiconductor capabilities and ... IIT- Madras director Prof V Kamakoti, who was in the national capital to attend the event, spoke to News18, on what makes the institute stick to the top every year, what it is that it strives to improve upon next, why it wants to beat IISc in the rankings, a positive trend in core jobs and why AI will create more jobs in the future. ... IIT Madras bagging the top rank for the sixth time in a row is a validation of the institute's dedicat

### Example 2

In [52]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS

In [53]:
from langchain_ollama import ChatOllama, OllamaEmbeddings

nomic_embeddings = OllamaEmbeddings(model='nomic-embed-text', base_url='http://localhost:11434')
llama_llm = ChatOllama(model='llama3.2:1b', base_url='http://localhost:11434', temperature=0)

loader = WebBaseLoader("https://docs.smith.langchain.com/overview")

docs = loader.load()

documents = RecursiveCharacterTextSplitter(chunk_size=1024, chunk_overlap=50).split_documents(docs)

vector = FAISS.from_documents(documents, nomic_embeddings)

retriever = vector.as_retriever()


In [54]:
from langchain.tools.retriever import create_retriever_tool
from langchain_community.tools import DuckDuckGoSearchRun

search = DuckDuckGoSearchRun()

retriever_tools = create_retriever_tool(
    retriever,
    "langsmith_search",
    "search for information about langsmith. For any question about langsmith you must add this tool"
)

tools = [search, retriever_tools]

In [55]:
prompt = ChatPromptTemplate.from_template('You are a helpful assistant: {query} agent_scratchpad:{agent_scratchpad}')

In [56]:
agent = create_tool_calling_agent(llama_llm, tools, prompt)

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True, return_intermediate_steps=True)

In [57]:
agent_executor.invoke({"query": "How can langchain help with testing?"})



> Entering new AgentExecutor chain...

Invoking: `duckduckgo_search` with `{'query': 'langchain testing'}`


Building Test Automation With Playwright and LangChain Agents from Scratch. Note: We will be not using The PlayWright Browser Toolkit that is a collection of tools that allow your agent to ... LangSmith. The ability to quickly and reliably evaluate your LLM application allows AI engineers to iterate with confidence. Many of the fastest moving and most successful teams we see have efficient testing and experimentation processes. This generally involves (1) setting up a dataset of inputs and (optionally) expected outputs, (2) defining ... Indeed LangChain's library of Toolkits for agents to use, listed on their Integrations page, are sets of Tools built by the community for people to use, which could be an early example of agent type libraries built by the community. Conclusion. Hope this was a useful introduction into getting you started building with agents in LangChain. It's 

{'query': 'How can langchain help with testing?',
 'output': 'Here is the JSON response for the given prompt:\n\n{\n  "duckduckgo_search": {\n    "parameters": {"query": "langchain testing"}\n  }\n}',
 'intermediate_steps': [(ToolAgentAction(tool='duckduckgo_search', tool_input={'query': 'langchain testing'}, log="\nInvoking: `duckduckgo_search` with `{'query': 'langchain testing'}`\n\n\n", message_log=[AIMessageChunk(content='', additional_kwargs={}, response_metadata={'model': 'llama3.2:1b', 'created_at': '2024-10-31T10:03:23.904711Z', 'message': {'role': 'assistant', 'content': '', 'tool_calls': [{'function': {'name': 'duckduckgo_search', 'arguments': {'query': 'langchain testing'}}}]}, 'done_reason': 'stop', 'done': True, 'total_duration': 1370283791, 'load_duration': 30780583, 'prompt_eval_count': 264, 'prompt_eval_duration': 732347000, 'eval_count': 21, 'eval_duration': 605469000}, id='run-54d7a021-69e1-4f55-a1e1-5a978698a5dd', tool_calls=[{'name': 'duckduckgo_search', 'args': {'